# Stable Baselines

Здесь мы очень вкратце посмотрим на пример использования библиотеки [Stable Baselines 3](https://github.com/DLR-RM/stable-baselines3). Эта библиотека содержит оттестированные реализации различных алгоритмов RL и хорошо взаимодействует с библиотекой OpenAI Gym. Она предоставляет интерфейс, позволяющий обучать RL-агентов с использованием совсем небольшого количества кода.

## Настройка окружения

In [1]:
#@title Set up environment

import sys, os

if 'google.colab' in sys.modules and not os.path.exists('.setup_complete'):
    # Note: enviroments like CartPole-v0 require a display to render. We need to install pyvirtualdisplay etc 
    # in order to render from these environments
    !apt-get -qq install -y xvfb
    !pip install -q pyvirtualdisplay stable-baselines3

    !touch .setup_complete

# This code creates a virtual display to draw game images on.
# It will have no effect if your machine has a monitor.
if 'DISPLAY' not in os.environ:
    from pyvirtualdisplay import Display

    # Start virtual display
    display = Display(visible=0, size=(1024, 768))
    display.start()

    os.environ['DISPLAY'] = f':{display.display}'

Selecting previously unselected package xvfb.
(Reading database ... 160815 files and directories currently installed.)
Preparing to unpack .../xvfb_2%3a1.19.6-1ubuntu4.9_amd64.deb ...
Unpacking xvfb (2:1.19.6-1ubuntu4.9) ...
Setting up xvfb (2:1.19.6-1ubuntu4.9) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
     |████████████████████████████████| 174kB 6.6MB/s 


## Запуск обучения

In [2]:
import gym
from stable_baselines3 import A2C
from stable_baselines3.common.env_util import make_vec_env

# Создаём 4 процесса, внутри каждого из которых будет работать среда CartPole-v1
vec_env = make_vec_env(lambda: gym.make("CartPole-v1"), n_envs=4)

# Создаём объект, который при помощи алгоритма A2C обучит
# полносвязную нейросеть (MlpPolicy) решать задачу CartPole-v1
model = A2C("MlpPolicy", vec_env, verbose=1)

# Запускаем обучение, в котором в каждом из 4 подпроцессов env.step() будет вызван 25000 раз
model.learn(total_timesteps=25000, log_interval=1000)

# Вот так можно сохранить модель
model.save("a2c_cartpole")

# Чтобы проверить, что загрузка модели работает, удалим её из памяти
del model

Using cpu device
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 161       |
|    ep_rew_mean        | 161       |
| time/                 |           |
|    fps                | 2532      |
|    iterations         | 1000      |
|    time_elapsed       | 7         |
|    total_timesteps    | 20000     |
| train/                |           |
|    entropy_loss       | -0.56     |
|    explained_variance | -0.000279 |
|    learning_rate      | 0.0007    |
|    n_updates          | 999       |
|    policy_loss        | 0.591     |
|    value_loss         | 2.45      |
-------------------------------------


## Визуализация результатов

В следующей ячейке код для визуализации скопирован из [предыдущего семинара](https://github.com/dniku/neural_nets_dpo/blob/master/week15/reinforcement_learning.ipynb).

In [3]:
from IPython.display import HTML
import matplotlib.animation
import matplotlib.pyplot as plt
%matplotlib inline

def animate_frames(frames):
    new_height = 2.2
    original_height = frames[0].shape[0]
    original_width = frames[0].shape[1]
    new_width = (new_height / original_height) * original_width
    fig = plt.figure(figsize=(new_width, new_height), dpi=120)
    
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    patch = ax.imshow(frames[0], aspect='auto', animated=True, interpolation='bilinear')
    animate = lambda i: patch.set_data(frames[i])
    
    ani = matplotlib.animation.FuncAnimation(fig, animate, frames=len(frames), interval=50)
    
    plt.close()
    return HTML(ani.to_jshtml())

Загружаем модель, создаём (теперь уже не параллельную) среду и делаем в ней не более 500 шагов, сохраняя картинки:

In [4]:
from tqdm.notebook import tqdm

model = A2C.load("a2c_cartpole")
env = gym.make('CartPole-v1')

obs = env.reset()
frames = [env.render(mode='rgb_array')]
for _ in tqdm(range(500)):
    action, _states = model.predict(obs)
    obs, _ , done, _ = env.step(action)
    frames.append(env.render(mode='rgb_array'))
    if done:
        break

Рисуем результат:

In [5]:
animate_frames(frames)